# Tutorial to the GraNNField Simulation Package

### 1. Data Extraction from AIMD

In [ ]:
from ase.io import read, write
from grannfield.data.data_from_files import extxyz_to_db

In [ ]:
dir = 'path_to_aimd_data'

In [ ]:
# Read AIMD trajectories
atoms = read(dir+'/vasprun.xml', index=':')

In [ ]:
# Create extxyz of AIMD trajectories
write('aimd.extxyz', atoms, append=True, format='extxyz')

In [ ]:
# Generate GraNNField database (.db) for model training
extxyz_to_db('aimd.extxyz', 'database.db')

## 2. Model Training

In [ ]:
import os
import torch

from torch.optim import Adam

from grannfield.data.materials import MaterialsData, ConcatMaterialsData
from grannfield.data.data import MaterialsLoader, create_subset, train_test_split

from grannfield.learn.trainer import Trainer
from grannfield.learn.hooks import CSVHook, ExponentialDecayHook, ReduceLROnPlateauHook

from grannfield.utils.statistics import Normalizer

In [ ]:
dir = 'path_to_database_directory'

In [ ]:
dataset = ConcatMaterialsData(MaterialsData(os.path.join(dir, db)))

In [ ]:
index_train = 'index_of_training'
index_val = 'index_of_val'

In [ ]:
train = create_subset(dataset, index_train)
val = create_subset(dataset, index_val)

In [ ]:
train_loader = MaterialsLoader(train, batch_size=2, shuffle=True)
val_loader = MaterialsLoader(val, batch_size=1)

In [ ]:
from grannfield.models.escn.escn import eSCN

model = eSCN(
    max_neighbors = 500,
    cutoff = 6.0,
    use_pbc = True,
    regress_forces = True,
    offset_degree = 1
)

In [ ]:
if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'

In [ ]:
optimizer = Adam(model.parameters(), lr=1E-3, weight_decay=1E-6, betas=(0.9, 0.999), amsgrad=True)

In [ ]:
from grannfield.learn.metrics import MeanAbsoluteError

metrics = [MeanAbsoluteError(target='energy', model_output='energy'),
           MeanAbsoluteError(target='forces', model_output='forces')]

In [ ]:
y_weight = 1
dy_weight = 1000

def loss(batch, result):
    diff_energy = batch['energy']-result['energy']
    err_sq_energy = torch.mean(diff_energy ** 2)

    diff_forces = batch['forces'] - result['forces']
    err_sq_forces = torch.mean(diff_forces ** 2)

    err_sq = y_weight * err_sq_energy + dy_weight * err_sq_forces

    return err_sq

In [ ]:
chk = './'

In [ ]:
hooks = [
    CSVHook(log_path=chk, metrics=metrics),
    ReduceLROnPlateauHook(
        optimizer,
        patience=5,
        min_lr=1E-6,
        factor=0.8,
        stop_after_min=True
    )
]

In [ ]:
normalizers = torch.load('path_to_normalizers')

In [ ]:
trainer = Trainer(
    model_path = chk,
    model = model,
    hooks = hooks,
    loss_fn = loss,
    optimizer = optimizer,
    train_loader = train_loader,
    checkpoint_interval = 5000,
    validation_loader = val_loader,
    validation_interval = len(train_loader),
    normalizers = normalizers,
    clip_grad_norm = None,
    ema_decay = 0.999,
    amp = False,
    early_stopping_time = 3600*10,
    device = device,
    print_freq = 1000
)

In [ ]:
n_epochs = 2000

In [ ]:
trainer.train(n_epochs=n_epochs)

## 3. Metrics Evaluation (Ionic Conductivity)

In [ ]:
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.diffusion.analyzer import DiffusionAnalyzer, get_arrhenius_plot, get_extrapolated_conductivity

In [ ]:
ase_to_pmg = AseAtomsAdaptor()

In [ ]:
diffusing_species = 'Li'
temperature = 300
time_step = 2
smoothed = False
steps_to_ignore = 15000
avg_nsteps = 1000
step_skip = 1

In [ ]:
diff_analyzer = DiffusionAnalyzer.from_structures('path_to_trajectories',
                                                           specie=diffusing_species,
                                                           temperature=temperatures,
                                                           time_step=time_step,
                                                           smoothed=smoothed,
                                                           step_skip=step_skip,
                                                           avg_nsteps=avg_nsteps)

In [ ]:
diffusivity = diff_analyzer.diffusivity

## 4. Molecular Dynamics (ASE)

In [ ]:
import os
import shutil
from datetime import datetime

import torch
import numpy as np

from pymatgen.io.ase import AseAtomsAdaptor
import seaborn as sns

from asap3.analysis.rdf import RadialDistributionFunction

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pymatgen.analysis.diffusion.analyzer import DiffusionAnalyzer, get_arrhenius_plot, get_extrapolated_conductivity

from grannfield.data.materials import MaterialsData

import time
from ase import units
from ase.io import read, write
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.velocitydistribution import Stationary, ZeroRotation

from grannfield.utils.md import NoseHoover

In [ ]:
if torch.cuda.is_available():
    device = 'cuda:1'
else:
    device = 'cpu'

In [ ]:
chk = './'

In [ ]:
from grannfield.models.escn.escn import eSCN

model = eSCN(
    max_neighbors = 500,
    cutoff = 6.0,
    use_pbc = True,
    regress_forces = True,
    offset_degree = 5
)

In [ ]:
normalizers = 'path_to_normalizers'

In [ ]:
model.load_state_dict('path_to_model_checkpoint')

In [ ]:
from grannfield.utils.calculator import GraNNFieldCalculator

calculator = GraNNFieldCalculator(base_model=model, device=device, energy='energy',forces='forces', normalizers = normalizers, apply_constraint=False)

In [ ]:
ase_to_pmg = AseAtomsAdaptor()

In [ ]:
def save_to_xyz(atoms, logdir, prefix=""):
    write(
        filename=os.path.join(os.path.join(logdir), prefix + '.xyz'),
        images=atoms,
        format="extxyz",
        append=True,
    )

In [ ]:
save_dir = 'path_for_saved_md_data'


at = read('path_to_input_structure')
at.set_calculator(calculator)

temperature = 'target_temperature'
timestep = 2 * units.fs
n_steps = 25000

MaxwellBoltzmannDistribution(atoms=at, temperature_K=temperature)

ZeroRotation(at)
Stationary(at)

log_frequency = 100
save_frequency = 10
log_dir = save_dir
prefix = 'prefix'

print(prefix)

save_to_xyz(at, logdir=log_dir, prefix=prefix)

nvt_dyn = NoseHoover(
atoms=at,
timestep=timestep,
temperature=temperature,
)

start = time.time()
for step in range(1, n_steps):
    nvt_dyn.run(steps=1)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    if not step % log_frequency:
        print("GRANNFIELD_MD: Step ", step)
        print("Elapsed Time [hh:mm:ss.ms]: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        print(f"Simulation Time [ps]: {timestep / units.fs * step / 1000}")
        print(f"Temperature [K]: {at.get_temperature()}")
        print(f"Total Energy [eV]: {at.get_potential_energy()}")
        print(f"Forces Max [eV/Å]: {at.get_forces().max()}\n")

    if not step % save_frequency:
        save_to_xyz(at, logdir=log_dir, prefix=prefix)

## 5. Molecular Dynamics 

Example LAMMPS input file:

units         metal
boundary      p p p
atom_style    atomic

pair_style    grannfield/gpu .

read_data     ./input.data

pair_coeff    * * model_checkpoint.pth.tar Ge Li P S

dump          1 all custom 1 output.lammpstrj id element x y z fx fy fz
dump_modify   1 sort id element Ge Li P S

thermo_style  custom step time cpu pe ke etotal temp
thermo        1
log           none

velocity      all create 800 12345
fix           1 all nvt temp 800 800 0.1
timestep      2.0e-3
run           25000
